In [ ]:
import MetaTrader5 as mt5
#Connect to MT5
mt5.initialize()



# Trading Bot Configuration
ticker = 'GBPUSD'
pip_size = 0.0001  # 1 pip for GBPUSD
qty = 0.16  # Lot Size

buy_order_type = mt5.ORDER_TYPE_BUY
sell_order_type = mt5.ORDER_TYPE_SELL

buy_price = mt5.symbol_info_tick("GBPUSD").ask
sell_price = mt5.symbol_info_tick("GBPUSD").bid

# Dictionaries
buy_signals = {'Buy_Signal': buy_price}
sell_signals = {'Sell_Signal': sell_price}

# Stop-Loss & Take-Profit for Buy Trades
buy_sl = buy_signals['Buy_Signal'] - (6 * pip_size)  # 6-pip SL
buy_tp = buy_signals['Buy_Signal'] + (12 * pip_size)  # 12-pip TP

# Stop-Loss & Take-Profit for Sell Trades
sell_sl = sell_signals['Sell_Signal'] + (6 * pip_size)  # 6-pip SL
sell_tp = sell_signals['Sell_Signal'] - (12 * pip_size)  # 12-pip TP


# Function to Create Either Buy or Sell Orders
def create_order(ticker, qty, order_type, price, sl, tp):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": qty,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "comment": "Python open position",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }
    
    # Send the trading request
    order = mt5.order_send(request)
    
    if order and order.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Order placed successfully: Ticket {order.order}")
        return order.order  # Return the trade ID
    else:
        print(f"Order failed: {order.comment if order else 'No response from server'}")
        return None


# Function for Closing Buy or Sell Orders
def close_order(ticker, qty, order_type, price):
    positions = mt5.positions_get(symbol=ticker)  # Get open positions for this ticker
    
    if positions:  # If there is an open trade
        position_id = positions[0]._asdict()['ticket']  # Get first open position's ticket
        
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": ticker,
            "volume": qty,
            "type": order_type,
            "price": price,
            "position": position_id,
            "comment": "Closing trade",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC
        }

        # Send the trading request
        order = mt5.order_send(request)
        
        if order and order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Position {position_id} closed successfully")
        else:
            print(f"Failed to close position {position_id}: {order.comment if order else 'No response from server'}")
    else:
        print(f"No open positions to close for {ticker}")

        

        

        
import time

#TRADINGBOT
#Pine Script

while True:
    symbol = "GBPUSD"
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 30)  # Fetch more historical data
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    
        # Apply detection functions
    df = detect_support_resistance(df, window=10)
    df = detect_breakout_retest(df)
    df = detect_trade_signals(df)
    
    # Ensure 'Stop_Loss' & 'Take_Profit' exist before using them
    if 'Stop_Loss' not in df.columns:
        df['Stop_Loss'] = np.nan  
    if 'Take_Profit' not in df.columns:
        df['Take_Profit'] = np.nan 
    
    df['Stop_Loss'].fillna(0, inplace=True)
    df['Take_Profit'].fillna(0, inplace=True)


    long_condition = (
        pd.notna(df['Resistance'].shift(2)).any()  # Resistance exists 2 candles ago
        and (df['close'].shift(1) > df['Resistance'].shift(2)).any()  # Breakout confirmed
        and (df['low'] <= df['Resistance'].shift(2)).any()  # Retest confirmation
    )
    short_condition = (
        pd.notna(df['Support'].shift(2)).any()  # Support exists 2 candles ago
        and (df['close'].shift(1) < df['Support'].shift(2)).any()  # Breakout confirmed
        and (df['high'] >= df['Support'].shift(2)).any()  # Retest confirmation
    )
    close_long_condition = (
        (df['close'] <= df['Stop_Loss']).any() |  # Stop-loss reached
        (df['close'] >= df['Take_Profit']).any()  # Take-profit reached
    )
    close_short_condition = (
        (df['close'] >= df['Stop_Loss']).any() |  # Stop-loss reached for short
        (df['close'] <= df['Take_Profit']).any()  # Take-profit reached for short
    )
    
    # Ensure 'Breakeven_Level' exists before using it
    if 'Breakeven_Level' not in df.columns:
        df['Breakeven_Level'] = np.nan  # Initialize with NaN to prevent KeyErrors

    # Fill missing Breakeven Levels with 0
    df['Breakeven_Level'].fillna(0, inplace=True)
    
    breakeven_buy_condition = (
        df['high'] >= df['Breakeven_Level']
    ).any()
    breakeven_sell_condition = (
        df['low'] <= df['Breakeven_Level']
    ).any()
    
    already_buy = False
    already_sell = False
    
    try:
        already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
        already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
        pass
    
    no_positions = len(mt5.positions_get()) == 0
    
    # 🟢 RECORD TRADE OPEN TIME HERE
    trade_open_time = None

    #Implementing conditions for longs and shorts
    if long_condition:
        if no_positions:
            create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
            print('Buy Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅
        if already_sell:
            close_order(ticker, qty, buy_order_type, buy_price)
            print('Sell Position Closed')
            time.sleep(1)
            create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
            print('Buy Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅

    if short_condition:
        if no_positions:
            create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
            print('Sell Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅
        if already_buy:
            close_order(ticker, qty, sell_order_type, sell_price)
            print('Buy Position Closed')
            time.sleep(1)
            create_order(ticker, qty, sell_order_type, sell_price, sell_sl, buy_tp)
            print('Sell Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅
    
    try:
        already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
        already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
        pass
    # 🟢 CHECK EXIT CONDITIONS ONLY AFTER HOLDING FOR 60 SECONDS ✅
    if trade_open_time and time.time() - trade_open_time > 60:
        if close_long_condition and already_buy:
            close_order(ticker, qty, sell_order_type, sell_price)
            print('Closed Buy Position')

        if close_short_condition and already_sell:
            close_order(ticker, qty, buy_order_type, buy_price)
            print('Closed Sell Position')
    
    already_buy = False
    already_sell = False
    
    #Implementing condition for breakeven
    # Initialize hit_be before using it
    hit_be = pd.DataFrame()  # Ensures it's always defined

    # Implementing condition for breakeven
    if not hit_be.empty and (hit_sl.empty or hit_be.iloc[0]['time'] < hit_sl.iloc[0]['time']):
        if trade_type == 'Buy':
            buy_breakevens += 1
        else:
            sell_breakevens += 1
    
    time.sleep(10)
